# **Assignment - 2**

---
### **Introduction**
---


This assignment focuses on solving a hill-climbing problem with the intent to comprehend how Tensorflow uses optimization techniques to minimize the cost (loss) function in a neural network. The objective is to utilize several Python-coded algorithms to determine the best feasible method for assigning packages to a set number of trucks. To discover the best potential solutions, three approaches were made: random package assignment, random optimization by truck swapping, and simulated annealing by truck swapping.

The first approach assigns packages randomly to a sorted list of trucks based on their capacities. Each truck is designed to have three deliveries with any number of packages (that does not exceed the truck's capacity) assigned to it. The algorithm keeps track of the unassigned packages to ensure all of them are delivered and only one package is assigned to a truck.

The random swapping of trucks approach is designed as an optimizer that further minimizes the cost function of the initial assignment. This is accomplished by iterating through the previously created assignments and swapping the packages allocated to it, whilst ensuring the weight of each of the swapped packages does not exceed the truck’s capacity.

The third approach makes use of the simulated annealing algorithm, a metaheuristic algorithm used to solve optimization problems. The algorithm starts with an initial solution and iteratively generates new candidate solutions by perturbing the current solution, and unlike greedy algorithms, it accepts some solutions that are worse than the current solution in order to avoid getting stuck in local optima. As the algorithm progresses, the acceptance of worse solutions is gradually reduced by decreasing the temperature, from the temperature starting off as initially high and is gradually lowered over time until the algorithm converges to a near-optimal solution (GeeksForGeeks 2023) 


In [1]:
# importing relevant libraries
import random
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

---
### **Task 1 and 2:**
---

A function, generate_data(), was created to generate the required 40 trucks with varying capacity between 500 to 1000 kg as well as 120 packages with different weights and distances, ranging from 200-1000 and 20-100 respectively. To generate the varying capacities and distances, the inbuilt function in the Numpy library, Random randint was made use of and the data produced was stored in two different dataframes for further use.

In [2]:
# create a function to generate trucks and packages dataframes
def generate_data():
    # generate 40 trucks with random capacity between 500 and 1000
    trucks_df = pd.DataFrame({'truck_id': range(1, 41), 'capacity': np.random.randint(500, 1001, 40)})
    # generate 120 packages with random weight between 200 and 1000 and random distance between 20 and 100
    packages_df = pd.DataFrame({'package_id': range(1, 121), 'weight': np.random.randint(200, max(trucks_df['capacity']), 120), 'distance': np.random.randint(20, 101, 120)})
    return trucks_df, packages_df

---
### **Task 3:**
---

For the initial assignment, assign_packages() and generate_init_data() functions were designed.

The generate_init_data() function is used as a helper method that calls both generate_data() and assign_packages() procedures. The two functions are executed in a loop until there is a perfect set of randomly generated trucks and packages and no unassigned packages.

The assign_packages() functions take the previously generated data to randomly assign packages to trucks. The passed packages dataframe is sorted according to their weights, for easier assignment. Further, The passed truck data is used to create a new dictionary that stores the number of packages held within the truck, which can be used to make sure that no more than 3 deliveries take place. Finally, a new dataframe, Temp_df is created to be used for tracking assignment with the IDs of the trucks and the packages they are carrying along with the delivery number.

For starting the initial assignment a while loop is created.The while loop first starts with checking the length of the packages dataframe to be more than 0, and all the truck delivery capacity to be equal to 3. For further assurance, using a try block code it made sure that no unassigned packages exceeded the truck capacity leading to an impossible assignment.

If assignment is possible, the function gets the capacity of the truck and creates a list to store the packages assigned to the truck. It then loops through the unassigned packages and assigns the packages to the truck, updating the truck’s capacity by subtracting the weight of the now assigned package. Additionally, using an if statement checks if the weight of the package is less than or equal to the capacity (remaining) of the truck. If the capacity of the truck is 0 or less than the minimum weight of the unassigned packages, the loop breaks. Further in the function, if any packages are assigned to the truck, the function updates the number of deliveries for the truck and adds the packages to the temp_df dataframe. In case the maximum capacity of the truck is exceeded, the extra packages are removed and stored into the unassigned_packages dataframe and the truck_deliveries dictionary is updated.

Finally, the function returns the temp_df dataframe containing the assigned packages and the unassigned_packages dataframe containing the packages that could not be assigned.


In [3]:
# function to assign packages to trucks based on the capacity of the truck and the weight of the package
def assign_packages(trucks_df, packages_df):
    # dataframe to store the assigned packages
    temp_df = pd.DataFrame(columns = ['truck_id', 'package_id', 'delivery_num'])
    # dictionary to store the number of deliveries for each truck
    truck_deliveries = {truck_id: 0 for truck_id in trucks_df['truck_id'].values}
    # sorting the packages in descending order of weight
    unassigned_packages = packages_df.sort_values(by = 'weight', ascending = False)
    # flag to check if assignment is complete/possible
    flag = False
    # loop until all packages are assigned or assignment is not possible
    while flag == False:
        # loop through the trucks in descending order of capacity
        for truck_id in trucks_df.sort_values(by = 'capacity', ascending = False)['truck_id'].values:
            # check if all packages are assigned
            if len(unassigned_packages) == 0:
                flag = True
                break
            # check if the truck has already made 3 deliveries
            if truck_deliveries[truck_id] == 3:
                continue
            # try block to handle the case when the minimum weight of the unassigned packages cannot be done
            try:
                # check if the minimum weight of the unassigned packages is greater than the capacity of the 
                # truck available in truck_deliveries, if yes, then assignment is not possible
                if min(unassigned_packages['weight'].values) > max(
                        trucks_df[trucks_df['truck_id'].isin(k for k, v in truck_deliveries.items() if v < 3)][
                            'capacity'].values):
                    flag = True
                    break
            except:
                # do nothing
                pass
            # get the capacity of the truck
            capacity = trucks_df[trucks_df['truck_id'] == truck_id]['capacity'].values[0]
            # list to store the packages assigned to the truck
            packages = []
            # loop through the unassigned packages and assign the packages to the truck
            for _, package in unassigned_packages.iterrows():
                # check if the weight of the package is less than the capacity (remaining) of the truck
                if package['weight'] <= capacity:
                    # add the package to the list of packages assigned to the truck
                    packages.append(package['package_id'])
                    # update the capacity of the truck
                    capacity -= package['weight']
                # check if the capacity of the truck is 0 or less than the minimum weight
                #  of the unassigned packages
                if capacity == 0 or capacity < min(unassigned_packages['weight'].values):
                    break
            # check if any packages are assigned to the truck
            if len(packages) > 0:
                # update the number of deliveries for the truck
                delivery_num = truck_deliveries[truck_id] + 1
                # add the packages to the dataframe
                for pack in packages:
                    temp_df = temp_df.append({'truck_id': truck_id, 'package_id': pack,
                                          'delivery_num': delivery_num}, ignore_index = True)
                    # remove the assigned packages from the unassigned packages dataframe
                    unassigned_packages = unassigned_packages[unassigned_packages['package_id'] != pack]
                # update the number of deliveries for the truck
                truck_deliveries[truck_id] += 1
    return temp_df, unassigned_packages

In [4]:
# create a function to generate trucks and packages dataframes and assign packages until all packages are assigned
def generate_init_data():
    # loop until all packages are assigned
    while True:
        # generate trucks and packages dataframes
        trucks_df, packages_df = generate_data()
        # assign packages
        assignment_df, unassigned_packages = assign_packages(trucks_df, packages_df)
        # check if all packages are assigned
        if len(unassigned_packages) == 0:
            break
    return trucks_df, packages_df, assignment_df

In [5]:
trucks_df, packages_df, assignment_df = generate_init_data()

In [6]:
# checking the number of trucks and packages
assignment_df.sort_values(by = 'truck_id').head(20)

,truck_id,package_id,delivery_num
69,1,63,2
29,1,61,1
94,2,64,3
46,2,117,2
6,2,93,1
93,2,29,3
25,3,103,1
65,3,6,2
104,4,53,3
105,4,84,3


In [7]:
# checking the number of deliveries for each truck
assignment_df.groupby('truck_id').value_counts().head(20).to_frame()

0
truck_id package_id delivery_num   
1        61         1             1
         63         2             1
2        117        2             1
         93         1             1
         64         3             1
         29         3             1
3        103        1             1
         6          2             1
4        84         3             1
         72         3             1
         53         3             1
         52         2             1
         51         1             1
5        90         1             1
         114        3             1
         105        2             1
         27         3             1
         5          3             1
6        75         2             1
         55         1             1

In [8]:
# checking if all packages are assigned
assignment_df['package_id'].value_counts()

12     1
4      1
30     1
13     1
109    1
      ..
81     1
101    1
98     1
77     1
20     1
Name: package_id, Length: 120, dtype: int64

---
### **Task 4:**
---

The cost of each assignment is calculated through the given formula,

> #### $Cost \ = \ \frac{Capacity \ of \ Truck \ - \ Weight \ of \ Package}{Distance \ of \ Package}$

To calculate the cost automatically for all the different trucks and packages, a function calculate_cost() was implemented. Using a for loop, each assignment is iterated to identify the truck capacity, package weight and distance. The sum of the cost of all assignments is then used to calculate the average cost of all the trucks assigned. 

In [9]:
def calculate_cost(assignment_df, trucks_df, packages_df):
    # cost of each assignment is (capacity of truck - weight of package) / distance of package
    assignment_df['cost'] = 0
    cost = 0
    costs = []
    for _, row in assignment_df.iterrows():
        truck_id = row['truck_id']
        package_id = row['package_id']
        capacity = trucks_df[trucks_df['truck_id'] == truck_id]['capacity'].values[0]
        weight = packages_df[packages_df['package_id'] == package_id]['weight'].values[0]
        distance = packages_df[packages_df['package_id'] == package_id]['distance'].values[0]
        # inserting the cost of the assignment in the dataframe where the assignment is made 
        assignment_df.loc[(assignment_df['truck_id'] == truck_id) & (assignment_df['package_id'] == package_id), 'cost'] = (capacity - weight) / distance
    # get average cost of all assignments
    cost = sum(assignment_df['cost']) / len(trucks_df)
    return cost, assignment_df

In [10]:
initial_cost, assignment_df = calculate_cost(assignment_df, trucks_df, packages_df)
print('Cost of the initial assignment: ', initial_cost)

Cost of the initial assignment:  14.433568233580289


In [11]:
assignment_df

,truck_id,package_id,delivery_num,cost
0,18,12,1,0.166667
1,39,4,1,0.027027
2,24,86,1,0.032258
3,12,76,1,0.687500
4,33,113,1,0.891892
...,...,...,...,...
115,23,42,3,12.612245
116,16,46,3,11.309091
117,16,47,3,7.090909
118,16,82,3,10.566667


---
### **Task 5-7:**
---

The optimize_cost function implements its optimization strategy on swapping the packages of a truck with another randomly selected truck. It takes in three dataframes: assignments_df containing the initial assignments of packages to trucks, trucks_df containing information about the trucks, and packages_df containing information about the packages.

The function starts by initializing two variables swap_no and flag. swap_no, that is used to keep track of the number of swaps made, while flag is a flag variable used to check if the assignment is complete or possible. The while loop runs until the ‘flag’ variable is set to True. Within the while loop, a for loop iterates through each row in the assignments_df dataframe, and creates a temporary copy, temp_df, one at a time for use. The algorithm randomly selects another truck from trucks_df using the random.choice() function. If the random truck id is the same as the truck id of the current assignment, to ensure the truck selected at random is not the same as the one in use. If the randomly selected truck has enough capacity to carry the package assigned to the current truck, and the package assigned to the randomly selected truck can fit in the current truck, then the function swaps the assignments of the two trucks. 

After swapping the assignments, the calculate_cost() function is called to calculate the cost of the new assignments. If the cost of the new assignment for the current truck is less than the cost of the old assignment, then the following values of the new assignment are printed: Swap number, Swapped package between the trucks with cost of the initial and new assignment, and the previous and new average costs of all assignments. After each swap, the flag variable is set to True to indicate that the assignment is completed, and the assignment_df dataframe is updated. If no swaps are made in an iteration, the ‘flag’ remains False, and the loop continues to the next iteration.

Once the assignment is complete, the function calculates the average cost of all assignments and returns it along with the updated assignments_df.


In [12]:
def optimize_cost(assignments_df, trucks_df, packages_df):
    # keeping track of swap number
    swap_no = 1
    # flag to check if assignment is complete/possible
    flag = False
    while flag == False:
        # loop through the assignments
        for _, row in assignments_df.iterrows():
            temp_df = assignments_df.copy()
            # get the truck id and package id
            truck_id = row['truck_id']
            package_id = row['package_id']
            # get random truck id 
            random_truck_id = random.choice(trucks_df['truck_id'].values)
            # check if the random truck id is same as the truck id of the assignment
            if random_truck_id == truck_id:
                continue
            # get the package id assigned to the random truck
            random_truck_package_id = temp_df[temp_df['truck_id'] == random_truck_id]['package_id'].values[0]
            # check if weight of the package is less than the capacity of the random truck
            if packages_df[packages_df['package_id'] == package_id]['weight'].values[0] < trucks_df[trucks_df['truck_id'] == random_truck_id]['capacity'].values[0]:
                # check if weight of the package assigned to the random truck is less than the capacity of the truck
                if packages_df[packages_df['package_id'] == random_truck_package_id]['weight'].values[0] < trucks_df[trucks_df['truck_id'] == truck_id]['capacity'].values[0]:                
                    # swap the packages
                    temp_df.loc[temp_df['truck_id'] == truck_id, 'package_id'] = random_truck_package_id
                    temp_df.loc[temp_df['truck_id'] == random_truck_id, 'package_id'] = package_id
                    # calculate the cost of the new assignment
                    cost, temp_df = calculate_cost(temp_df, trucks_df, packages_df)
                    # check if the cost of new assignment for truck_id is less than the cost of the old assignment
                    if temp_df[temp_df['truck_id'] == truck_id]['cost'].values[0] < assignments_df[assignments_df['truck_id'] == truck_id]['cost'].values[0]:
                        # print the swap details
                        print('='*100)
                        print('Swap no.: {} ==> Swap for truck {} and {} '.format(swap_no, truck_id, random_truck_id))
                        print('-'*100)
                        print('Swapped package {} from truck {} to truck {}'.format(package_id, truck_id, random_truck_id))
                        print('Cost of initial assignment: ', assignments_df[assignments_df['truck_id'] == truck_id]['cost'].values[0])
                        print('Cost of new assignment: ', temp_df[temp_df['truck_id'] == truck_id]['cost'].values[0])
                        print('*'*100)
                        print('Swapped package {} from truck {} to truck {}'.format(random_truck_package_id, random_truck_id, truck_id))
                        print('Cost of initial assignment: ', assignments_df[assignments_df['truck_id'] == random_truck_id]['cost'].values[0])
                        print('Cost of new assignment: ', temp_df[temp_df['truck_id'] == random_truck_id]['cost'].values[0])
                        print('-'*100)
                        print('Previous average cost of all assignments: ', sum(assignments_df['cost']) / len(trucks_df))
                        print('New average cost of all assignments: ', sum(temp_df['cost']) / len(trucks_df))
                        print('='*100)
                        print()
                        # update the assignments dataframe
                        assignments_df = temp_df.copy()
                        swap_no += 1
                        flag = True
        return sum(assignments_df['cost']) / len(trucks_df), assignments_df

In [13]:
optimized_cost, optimized_assignment_df = optimize_cost(assignment_df, trucks_df, packages_df)

Swap no.: 1 ==> Swap for truck 7 and 37 
----------------------------------------------------------------------------------------------------
Swapped package 45 from truck 7 to truck 37
Cost of initial assignment:  0.21568627450980393
Cost of new assignment:  0.19402985074626866
****************************************************************************************************
Swapped package 97 from truck 37 to truck 7
Cost of initial assignment:  0.14925373134328357
Cost of new assignment:  0.1568627450980392
----------------------------------------------------------------------------------------------------
Previous average cost of all assignments:  14.433568233580289
New average cost of all assignments:  14.35275325211507

Swap no.: 2 ==> Swap for truck 12 and 5 
----------------------------------------------------------------------------------------------------
Swapped package 7 from truck 12 to truck 5
Cost of initial assignment:  0.6875
Cost of new assignment:  0.52631578947368

---
### **Task 8:**
---

The simulated_annealing() function implements an annealing algorithm to find the best solution to the truck assignment problem. The function takes four inputs: assignments_df, trucks_df, and packages_df, which are dataframes containing the current assignments, trucks, and packages, respectively, and three initialized inputs: initial_temp, which sets the initial temperature for the algorithm (default is 1.0), temp_min, which sets the minimum temperature at which the algorithm should stop (default is 0.00001), and alpha, which sets the cooling factor for the temperature (default is 0.9).

The function uses a while loop to run the simulated annealing algorithm until the temperature cools down to the minimum temperature. Within the while loop, a for loop iterates through each row in the assignments_df dataframe, and creates a temporary copy, temp_df, one at a time for use. Then, the function randomly selects a different truck, making sure that the random selection is not the same as the one already in use. If the randomly selected truck has enough capacity to carry the package assigned to the current truck, and the package assigned to the randomly selected truck can fit in the current truck, then the function swaps the assignments of the two trucks. After swapping the assignments, the calculate_cost() function is called to calculate the cost of the new assignments. 

If the cost of the new assignment for the current truck is less than the cost of the old assignment, then the new assignment is accepted and becomes the current assignment. Otherwise, the function calculates an acceptance probability based on the difference in cost between the old and new assignments, and the current temperature. If a randomly generated number is less than this acceptance probability, then the new assignment is accepted.

Once all rows in the assignments_df have been checked, the temperature is decreased by multiplying it by the cooling factor alpha. The function then loops again until the temperature cools down to the minimum temperature. Finally, the function returns the average cost per truck and the final assignments_df dataframe with the optimized assignments.


In [14]:
# function to minimize the cost of the assignment using simulated annealing algorithm
def simulated_annealing(assignments_df, trucks_df, packages_df, initial_temp = 1.0, temp_min = 0.00001, alpha = 0.9):
    # loop until the system has cooled
    while initial_temp > temp_min:
        # loop through the assignments
        for _, row in assignments_df.iterrows():
            temp_df = assignments_df.copy()
            # get the truck id and package id
            truck_id = row['truck_id']
            package_id = row['package_id']
            # get random truck id 
            random_truck_id = random.choice(trucks_df['truck_id'].values)
            # check if the random truck id is same as the truck id of the assignment
            if random_truck_id == truck_id:
                continue
            # get the package id assigned to the random truck
            random_truck_package_id = temp_df[temp_df['truck_id'] == random_truck_id]['package_id'].values[0]
            # check if weight of the package is less than the capacity of the random truck
            if packages_df[packages_df['package_id'] == package_id]['weight'].values[0] < trucks_df[trucks_df['truck_id'] == random_truck_id]['capacity'].values[0]:
                # check if weight of the package assigned to the random truck is less than the capacity of the truck
                if packages_df[packages_df['package_id'] == random_truck_package_id]['weight'].values[0] < trucks_df[trucks_df['truck_id'] == truck_id]['capacity'].values[0]:                
                    # swap the packages
                    temp_df.loc[temp_df['truck_id'] == truck_id, 'package_id'] = random_truck_package_id
                    temp_df.loc[temp_df['truck_id'] == random_truck_id, 'package_id'] = package_id
                    # calculate the cost of the new assignment
                    cost, temp_df = calculate_cost(temp_df, trucks_df, packages_df)
                    # check if the cost of new assignment for truck_id is less than the cost of the old assignment
                    if temp_df[temp_df['truck_id'] == truck_id]['cost'].values[0] < assignments_df[assignments_df['truck_id'] == truck_id]['cost'].values[0]:
                        # update the assignments dataframe
                        assignments_df = temp_df
                    else:
                        # calculate the acceptance probability
                        ap = math.exp((assignments_df[assignments_df['truck_id'] == truck_id]['cost'].values[0] - temp_df[temp_df['truck_id'] == truck_id]['cost'].values[0]) / initial_temp)
                        # generate a random number
                        random_number = random.random()
                        # check if the random number is less than the acceptance probability
                        if random_number < ap:
                            # update the assignments dataframe
                            assignments_df = temp_df
        # cool the system
        initial_temp = initial_temp * alpha
    return sum(assignments_df['cost']) / len(trucks_df), assignments_df

In [15]:
# get the best assignment using simulated annealing algorithm
best_cost, best_assignment_df = simulated_annealing(assignment_df, trucks_df, packages_df)
best_cost

---
### **Conclusion**
---

Analyzing the three approaches implemented, the minimum cost deduced is given by the simulated annealing algorithm followed by the random swap of trucks optimizer. The inital average cost of all assignments resulted to be **14.43** which was then improved by optimizing using random swap, to **13.43**. Finally resulting in the least cost function, the simulated annealing function produced an average cost of **3.27**.

In [16]:
print('Initial cost: ', initial_cost)
print('Optimized cost: ', optimized_cost)
print('Best cost: ', best_cost)

Initial cost:  14.433568233580289
Optimized cost:  13.433467333889507
Best cost:  3.2730256313946393


---
### **References**
---

- GeeksForGeeks 2023, Simulated Annealing, GeeksForGeeks, <https://www.geeksforgeeks.org/simulated-annealing/>